数据集 CIFAR-10 python版

In [1]:
import _pickle as cPickle
import numpy as np
import os

In [2]:
CIFAR_DIR = './cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [3]:
with open(os.path.join(CIFAR_DIR, 'data_batch_1'), 'rb') as f:
    data = cPickle.load(f,encoding='iso-8859-1')
    print(type(data))
    print(data.keys())
    print(type(data['data']))
    print(type(data['labels']))
    print(type(data['batch_label']))
    print(data['data'].shape) # 10000个图片， 一张图片数据32 * 32 = 1024 * 3 = 3072 
    print(data['data'][0:2])  # RR--GG--BB 三个通道加在一起就是3072
    print(data['labels'][0:2])# 图片类型类型，数字表示
    print(data['batch_label'])#5个训练数据中的第一个 
    print(data['filenames'][0:2]) #图片名字
   

<class 'dict'>
dict_keys(['batch_label', 'labels', 'data', 'filenames'])
<class 'numpy.ndarray'>
<class 'list'>
<class 'str'>
(10000, 3072)
[[ 59  43  50 ... 140  84  72]
 [154 126 105 ... 139 142 144]]
[6, 9]
training batch 1 of 5
['leptodactylus_pentadactylus_s_000004.png', 'camion_s_000148.png']


In [4]:
image_arr = data['data'][100]
image_arr = image_arr.reshape((3,32,32)) # 32 32 3
image_arr = image_arr.transpose((1,2,0))
import matplotlib.pyplot as plt
#plt.imshow(image_arr)

构建模型

In [5]:
import tensorflow as tf

In [6]:
def load_data(filename):
    """ read data from data file"""
    with open(filename, 'rb') as f:
        data = cPickle.load(f,encoding='iso-8859-1')
        return data['data'], data['labels']

In [7]:
x = tf.placeholder(tf.float32, [None, 3072])
# [None]
y = tf.placeholder(tf.int64, [None])
# (3072 ,1)
w = tf.get_variable('w', [x.get_shape()[-1], 1], initializer = tf.random_normal_initializer(0,1)) #它会去搜索变量名，有就直接用，没有再新建
# (1, )
b = tf.get_variable('b', [1], initializer=tf.constant_initializer(0.0))
# [None, 3072] * [3072, 1]  = [None, 1]
y_ = tf.matmul(x,w) + b
# [None, 1]       计算公式 y = 1/(1+exp(-wx))  计算结果为1或0，用于二分类   exp(x)代表e的x次幂
p_y_1 = tf.nn.sigmoid(y_) #使用sigmoid模型算出y为1的概率，返回矩阵
# [None, 1]
y_reshape = tf.reshape(y, (-1, 1))
y_reshape_float = tf.cast(y_reshape, tf.float32)
loss = tf.reduce_mean(tf.square(y_reshape_float - p_y_1))
#boolean
predict = p_y_1 > 0.5 #大于0.5我们认为为1
# [1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(tf.cast(predict, tf.int64), y_reshape)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))
#train是修改计算图中tf.Variable（默认是计算图中所有tf.Variable，可以通过var_list指定）达到训练的目的
with tf.name_scope('train_op'):  #防止命名冲突,在该scope的命名的变量都是在train_op命名空间里的
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
Colocations handled automatically by placer.


In [28]:
#tensorflow.Dataset
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            for item, label in zip(data, labels):
                if label in [0,1]: #获取两个类的数据，二元分类
                    all_data.append(item)
                    all_labels.append(label)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 -1 # 每个数据之前是[0,255],缩放数据 [-1,1]之间，归一化,防止数据偏向一方
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            pass
    
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size exmples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception('have no more examples.')
        if end_indicator > self._num_examples:
           raise Exception('batch size is lager than all examples.')
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
        pass
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d'%i)for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)
# batch_data, batch_labels = train_data.next_batch(10)
# print(batch_data)
# print(batch_labels)

(10000, 3072)
(10000,)
(2000, 3072)
(2000,)


In [29]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
        [loss, accuracy, train_op],
            feed_dict = {
                x:batch_data,
                y:batch_labels
            })
        if i % 500 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' \
                 % (i, loss_val, acc_val))
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print ('[Test] Step: %d, acc %4.5f' %( i+1, test_acc))

[Train] Step: 0, loss: 0.80246, acc: 0.10000
[Train] Step: 500, loss: 0.19654, acc: 0.80000
[Train] Step: 1000, loss: 0.30233, acc: 0.70000
[Train] Step: 1500, loss: 0.15000, acc: 0.85000
[Train] Step: 2000, loss: 0.24091, acc: 0.75000
[Train] Step: 2500, loss: 0.10000, acc: 0.90000
[Train] Step: 3000, loss: 0.16493, acc: 0.80000
[Train] Step: 3500, loss: 0.19699, acc: 0.80000
[Train] Step: 4000, loss: 0.29898, acc: 0.70000
[Train] Step: 4500, loss: 0.19314, acc: 0.80000
(2000, 3072)
(2000,)
[Test] Step: 5000, acc 0.79200
[Train] Step: 5000, loss: 0.22580, acc: 0.75000
[Train] Step: 5500, loss: 0.09996, acc: 0.90000
[Train] Step: 6000, loss: 0.10001, acc: 0.90000
[Train] Step: 6500, loss: 0.20610, acc: 0.80000
[Train] Step: 7000, loss: 0.10437, acc: 0.90000
[Train] Step: 7500, loss: 0.09180, acc: 0.90000
[Train] Step: 8000, loss: 0.10004, acc: 0.90000
[Train] Step: 8500, loss: 0.08750, acc: 0.90000
[Train] Step: 9000, loss: 0.19998, acc: 0.80000
[Train] Step: 9500, loss: 0.14581, acc: 